### Use K means with k = 3 and perform clustering

In [32]:
import pandas as pd
import numpy as np
import warnings
np.warnings = warnings

df_clustering_name = f'INTERPRETABLE_xmeans_TOXIC_NON_TOXIC_DIVISION_embeddings_CONCAT_3.csv' #this contains the clustering results
df_classification_name = f'INTERPRETABLE_xmeans_TOXIC_NON_TOXIC_DIVISION_interpretable_CONCAT.csv' #this contains the classification results

df_clustering = pd.read_csv(df_clustering_name)
df_classification = pd.read_csv(df_classification_name)

#The same process below can  be repeated considering averaged embeddings instead of conctenation and using the inteprretable features directly

#df_clustering_name = f'INTERPRETABLE_xmeans_TOXIC_NON_TOXIC_DIVISION_embeddings_AVG_3.csv'
#df_classification_name = f'INTERPRETABLE_xmeans_TOXIC_NON_TOXIC_DIVISION_interpretable_AVG_3.csv'


In [22]:
df_classification_toxic = df_classification[df_classification.TOXICITY == 'toxic'].drop(columns = ['CLUSTER', 'TOXICITY'])
df_clustering_toxic = df_clustering[df_clustering.TOXICITY == 'toxic'].drop(columns = ['CLUSTER', 'TOXICITY'])

df_classification_safe = df_classification[df_classification.TOXICITY != 'toxic'].drop(columns = ['CLUSTER', 'TOXICITY'])
df_clustering_safe = df_clustering[df_clustering.TOXICITY != 'toxic'].drop(columns = ['CLUSTER', 'TOXICITY'])

In [ ]:
random_state = 42

initial_centers = kmeans_plusplus_initializer(df_clustering_toxic.values, 3, random_state  = random_state).initialize()

centroid_clustering_method_instance = kmeans(df_clustering_toxic.values, initial_centers, random_state = random_state )
centroid_clustering_method_instance.process()

clusters = centroid_clustering_method_instance.get_clusters() 
centers = centroid_clustering_method_instance.get_centers()

inertia = calculate_inertia(df_clustering_toxic.values, clusters, centers)
sil = calculate_silhouette(df_clustering_toxic.values, clusters, centers)
predictions = np.empty(len(df_clustering_toxic.values), dtype=int)


for cluster_index, cluster in enumerate(clusters):

    predictions[cluster] = cluster_index


In [ ]:
preds_toxic = []
for index in range(len(df_clustering_toxic.values)):
    for i, cluster in enumerate(clusters):
        if index in cluster:
            preds_toxic += [str(i) +'_tox']
            break


In [ ]:
random_state = 42

initial_centers = kmeans_plusplus_initializer(df_clustering_safe.values, k, random_state  = random_state).initialize()

centroid_clustering_method_instance = kmeans(df_clustering_safe.values, initial_centers, random_state = random_state )
centroid_clustering_method_instance.process()

clusters = centroid_clustering_method_instance.get_clusters() 
centers = centroid_clustering_method_instance.get_centers()

inertia = calculate_inertia(df_clustering_safe.values, clusters, centers)
sil = calculate_silhouette(df_clustering_safe.values, clusters, centers)
predictions = np.empty(len(df_clustering_safe.values), dtype=int)

for cluster_index, cluster in enumerate(clusters):
   
    predictions[cluster] = cluster_index



In [ ]:
preds_safe = []
for index in range(len(df_clustering_safe.values)):
    for i, cluster in enumerate(merged_clusters):
        if index in cluster:
            preds_safe += [str(i) +'_safe']
            break


In [ ]:
# Concatenate the two DataFrames
df_complete = pd.concat([df_clustering_toxic, df_clustering_safe], axis=0, ignore_index=True)
df_complete['cluster_preds'] = list(preds_toxic) + list(preds_safe)

dict_toxic_safe = {k: i for i, k in enumerate(set(df_complete['cluster_preds']))}

df_complete.to_csv('clustering_concat_results.csv')